Fetch information about a specific person identified by his/her ORCID ID, and handle API errors.

[Download Notebook](https://github.com/researchgraph/augment-api-beta/blob/main/docs/notebooks/affiliations.ipynb)

In [505]:
import sys
sys.path.append('../')

# !{sys.executable} -m pip install folium

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import ast
import altair as alt
import networkx as nx
import nx_altair as nxa
from wordcloud import WordCloud, STOPWORDS 
import pandas as pd
from datetime import datetime, date
import requests
import json
import folium

import os
from os.path import join, dirname
from dotenv import load_dotenv
load_dotenv();

### ORCID ID not found

In [506]:
# ORCID ID not found
API_KEY = os.environ.get("API_KEY")
ORCID = "0000-0003-XXXX-XXXX"

url = f'https://f130.azure-api.net/v1/orcid/{ORCID}?subscription-key={API_KEY}'
r = requests.get(url)

# print a short confirmation on completion
print('Augment API query complete ', r.status_code)

if r.status_code == 400:
    print(r.json()["message"])

Augment API query complete  400
FAILED: Invalid ORCID 0000-0003-XXXX-XXXX


### Missing API_KEY

In [507]:
# Missing API_KEY
API_KEY = ''
ORCID = "0000-0002-0715-6126"

url = f'https://f130.azure-api.net/v1/orcid/{ORCID}?subscription-key={API_KEY}'
r = requests.get(url)

# print a short confirmation on completion
print('Augment API query complete ', r.status_code)

if r.status_code == 401:
    print(f'Authentication error.')

Augment API query complete  401
Authentication error.


### ORCID ID does exist

In [508]:
# ORCID ID does exist
API_KEY = os.environ.get("API_KEY")
ORCID = "0000-0002-0068-716X"

url = f'https://f130.azure-api.net/v1/orcid/{ORCID}?subscription-key={API_KEY}'
r = requests.get(url)

# print a short confirmation on completion
print('Augment API query complete ', r.status_code)

if r.status_code == 200 and r.json()[0]["nodes"]["researchers"]:    
    researchers = r.json()[0]["nodes"]["researchers"]
    
    researcher = None
    for i in range(len(researchers)):
        if researchers[i]["orcid"] == ORCID:
            researcher = researchers[i]

print()
print(f'ORCID: {researcher["orcid"]}')
print(f'First name: {researcher["first_name"]}')
print(f'Last name: {researcher["last_name"]}')
print()
print(f'The researcher {researcher["full_name"]} is connected to {r.json()[0]["stats"]}.')

Augment API query complete  200

ORCID: 0000-0002-0068-716X
First name: Cameron
Last name: Neylon

The researcher Cameron Neylon is connected to {'datasets': 0, 'grants': 0, 'organisations': 207, 'publications': 157, 'researchers': 126}.


### List of co-authors
Only includes co-authors with ORCID IDs.

In [509]:
rf = pd.DataFrame(r.json()[0]["nodes"]["researchers"], columns=['first_name', 'last_name', 'full_name', 'orcid'])
dfStyler = rf.style.set_properties(**{'text-align': 'left'})
dfStyler.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])

,first_name,last_name,full_name,orcid
0,Patricia,Cruse,Patricia Cruse,0000-0002-9300-5278
1,John,Chodacki,John Chodacki,0000-0002-7378-2408
2,Anthony,Caravaggi,Anthony Caravaggi,0000-0002-1763-8970
3,Alkim,Ozaygen,Alkim Ozaygen,0000-0001-6813-8362
4,Susana,Teixeira,Susana Teixeira,0000-0002-6603-7936
5,Gabriel,Cavalli,Gabriel Cavalli,0000-0002-2856-1956
6,Judith,Peters,Judith Peters,0000-0001-5151-7710
7,Damian,Pattinson,Damian Pattinson,0000-0002-2606-6599
8,Julien,Colomb,Julien Colomb,0000-0002-3127-5520
9,Matthew,Blakeley,Matthew Blakeley,0000-0002-6412-4358


### List of co-author affiliations

In [510]:
# Strip wikidata ID from key
def force_wikidata(n):
    n['key'] = n['key'].split('/')[-1]
    return n

json = map(force_wikidata, r.json()[0]["nodes"]["organisations"])
of = pd.DataFrame(json, columns=['name', 'country', 'key', 'ror', 'lat', 'lon'])
of = of.rename(columns={'key': 'wikidata'})
dfStyler = of.style.set_properties(**{'text-align': 'left'})
dfStyler.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])

data = []
try:
    for index, row in of.iterrows():
        url = 'https://api.ror.org/organizations?query=' + row['wikidata']
        r2 = requests.get(url)

        # print an error message if status code != 200
        if r2.status_code != 200:
            print('ROR API query returned an error', r2.status_code)
        
        name = row['name']
        country = row['country']
        wikidata = row['wikidata']
        
        if r2.json()['number_of_results'] == 0:
            # we need to work on better aligning with ROR. Main issue seems to be wikidata identifiers for departments which ROR does not support
            print('No ROR record found for wikidata ' + row['name'] + ' ' + row['wikidata'])
        else:
            ror = r2.json()['items'][0]['id'][8:]
            lat = r2.json()['items'][0]['addresses'][0]['lat']
            lon = r2.json()['items'][0]['addresses'][0]['lng']
        data.append([name, country, wikidata, ror, lat, lon])
except IndexError as error:
    print('An IndexError has occured for index: ' + index)
    
of2 = pd.DataFrame(data, columns=['name', 'country', 'wikidata', 'ror', 'lat', 'lon'])   
dfStyler = of2.style.set_properties(**{'text-align': 'left'})
dfStyler.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])

No ROR record found for wikidata Göttingen State and University Library Q564783
No ROR record found for wikidata University of Toronto Faculty of Arts and Science Q7896481
No ROR record found for wikidata Swansea University Medical School Q7653715
No ROR record found for wikidata University of Toronto Faculty of Information Q7896482
No ROR record found for wikidata SURF (Samenwerkende Universitaire Rekenfaciliteiten) Q2422744
No ROR record found for wikidata Harvard College Q49123
No ROR record found for wikidata New York University in London Q99285271
No ROR record found for wikidata Kew Gardens Q188617
No ROR record found for wikidata Simon Fraser University - Vancouver Q99284970
No ROR record found for wikidata Paris Descartes University Q1155944
No ROR record found for wikidata Harvard Medical School Q49121


,name,country,wikidata,ror,lat,lon
0,Maastricht University,NL,Q1137652,ror.org/02jz4aj89,50.845833,5.687222
1,United States Environmental Protection Agency,US,Q460173,ror.org/03tns0030,38.893743,-77.030436
2,Science and Technology Facilities Council,GB,Q1669366,ror.org/057g20z61,51.566531,-1.785539
3,University of Missouri,US,Q579968,ror.org/02ymw8z06,38.939179,-92.331858
4,University of Illinois at Urbana–Champaign,US,Q457281,ror.org/047426m28,40.102182,-88.227194
5,Bangor University,GB,Q790029,ror.org/006jb1a24,53.228900,-4.130100
6,Uppsala University,SE,Q185246,ror.org/048a87296,59.851051,17.629966
7,Göttingen State and University Library,DE,Q564783,ror.org/048a87296,59.851051,17.629966
8,University of Strathclyde,GB,Q1859178,ror.org/00n3w3b69,55.861717,-4.241700
9,Wellcome Sanger Institute,GB,Q1142544,ror.org/05cy4wa09,52.079948,0.185566


In [513]:
# map affiliations on a world map, center around home institution (Monash University, for now done manually)
m = folium.Map(tiles='cartodbpositron', location=[of2.loc[[93]].lat, of2.loc[[93]].lon], zoom_start=4)

#Adding markers to the map
for index, row in of2.iterrows():
    folium.CircleMarker(location=[row['lat'], row['lon']],popup=row['name'], fill=True,
    color="darkgreen", radius=2).add_to(m)
m

In [512]:
# Generate a graph from the co-authors and their affiliations
# use XKCD colors: https://xkcd.com/color/rgb/
G = nx.Graph()

for index, row in rf.iterrows():
    G.add_node(row['orcid'], name=row['full_name'], color='xkcd:turquoise')

for index, row in of2.iterrows():
    G.add_node(row['wikidata'], name=row['name'], color='xkcd:vivid purple')

# Convert from and to for researcher relationships into ORCID IDs (to map the node labels)
def force_pid(n):
    n['from'] = n['from'].split('/')[-1]
    n['to'] = n['to'].split('/')[-1]
    return n

json = map(force_pid, r.json()[0]['relationships']['researcher-researcher'])
ef = pd.DataFrame(json, columns=['from', 'to'])

json = map(force_pid, r.json()[0]['relationships']['researcher-organisation'])
eo = pd.DataFrame(json, columns=['from', 'to'])

G.add_edges_from(ef.to_numpy())
G.add_edges_from(eo.to_numpy())
    
# Compute positions for viz.
pos = nx.spring_layout(G)

options = {
    "font_size": 18,
    "node_size": 50,
    "edge_color": "gray",
    "linewidths": 0.1,
    "width": 1
}

# Show information about the graph
print(nx.info(G))
print("Network density:", nx.density(G))

# Draw the graph using Altair
viz = nxa.draw_networkx(G, pos=pos, node_tooltip='name', node_color='color', **options).properties(width=800, height=800)

nx.write_gexf(G, "affiliations.gexf")

# Show it as an interactive plot!
viz.interactive()

Graph with 333 nodes and 417 edges
Network density: 0.007543688266579833


alt.LayerChart(...)